### NN

In [1]:
import os
import pandas as pd
import numpy as np
import imageio
import random
import torch
import torch.nn as nn

In [2]:
train_path = './train_data'
df_train = pd.read_csv('./df_train.csv')

val_path = './validation_data'
df_val = pd.read_csv('./df_validation.csv')

small_path = './small_sample'
df_small = pd.read_csv('./df_small.csv')

In [3]:
def read_batch_images(images, df, path):
    features_target = []
    for im in images:
        current_image = np.array(imageio.imread(os.path.join(path, im)))/255
        image_id = int(im.split('.')[0])
        target = int(df[df.id == image_id]['broken'])
        features_target.append((current_image, target))
    return features_target

In [24]:
def batch_generator(df, path, batch_size):
    images_name = [f for f in os.listdir(path) if f.split('.')[1] == 'png']
    random.shuffle(images_name)
    n = len(df)
    for i in range(0, n, batch_size):
        images_filter = images_name[i:(i+batch_size)]
        data = read_batch_images(images_filter, df, path)
        yield data

In [25]:
class BasicNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv2d(1, 32, 3)
        self.conv2 = nn.Conv2d(32, 16, 3)
        self.conv3 = nn.Conv2d(16, 8, 3)
        self.fc1 = nn.Linear(12800,512)
        self.fc2 = nn.Linear(512, 128)
        self.fc3 = nn.Linear(128, 1)
        self.pool = nn.MaxPool2d(2)
        self.relu = nn.ReLU()
        self.drp1 = nn.Dropout(0.25)
        self.drp2 = nn.Dropout(0.25)
        
    def forward(self, x):
        fo = self.pool(self.relu(self.conv1(x)))
        so = self.pool(self.relu(self.conv2(fo)))
        to = self.pool(self.relu(self.conv3(so))).view(-1,12800)
        fc1_out = self.drp1(self.relu(self.fc1(to)))
        fc2_out = self.drp2(self.relu(self.fc2(fc1_out)))
        out = self.fc3(fc2_out)
        return out

In [26]:
def train(model, optimizer, criterion, df_train, train_path, df_val, val_path, num_epochs, batch_size, device):
    for i in range(num_epochs):
        total_loss = 0
        model.train()
        print("epoch " + str(i))
        for batch in batch_generator(df_train, train_path, batch_size):
            x = [d[0] for d in batch]
            print(len(x))
            y = [d[1] for d in batch]
            n_batch_size = len(x) if len(x) < batch_size else batch_size
            x_t = torch.tensor(x, dtype=torch.float, device=device).view(n_batch_size, 1, 340, 340)
            y_t = torch.tensor(y, dtype=torch.float, device=device)
            model.zero_grad()
            output = model(x_t)
            loss = criterion(output.view(-1), y_t)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        val_probs, ys, val_loss = validate(model, criterion, val_data, batch_size, device)
        val_losses += [val_loss / len(val_data)]
        tr_losses += [total_loss / len(train_data)]
        print('Epoch {}, avg train loss per image {}, avg valid loss per image {}'.format(
            i+1, tr_losses[-1], val_losses[-1]
        ))
    return total_loss, val_loss

def validate(model, criterion, df_val, val_data, batch_size, device):
    model.eval()
    val_total_loss = 0
    probs = []
    ys = []
    sigmoid = torch.nn.Sigmoid()
    with torch.no_grad():
        for batch in batch_generator(df_val, val_data, batch_size):
            x = [d[0] for d in batch]
            y = [d[1] for d in batch]
            n_batch_size = len(x) if len(x) < batch_size else batch_size
            x_t = torch.tensor(x, dtype=torch.float, device=device).view(n_batch_size, 1, 340, 340)
            y_t = torch.tensor(y, dtype=torch.float, device=device)
            output = model(x_t)
            probs += list(sigmoid(output).view(-1).detach().cpu().numpy())
            ys += y
            loss = criterion(output.view(-1), y_t)
            val_total_loss += loss.item()
        return probs, ys, val_total_loss

In [27]:
epochs = 10
batch_size = 4
learning_rate = 0.001
device = torch.device('cuda')
model = BasicNet().to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCEWithLogitsLoss(reduction='sum')

In [28]:
train(model, optimizer, criterion, df_small, './small_sample', df_small, './small_sample', num_epochs=epochs, batch_size=4, device=device)

epoch 0
4
4
4
4
4
4
4
4
4
4
4
4
2


NameError: name 'val_data' is not defined

In [10]:
d = [[1,2], [5,4]]
dim(d)

NameError: name 'dim' is not defined